## Chain of thought experiments

En esta notebook hacemos unos pequeños experimentos con CoT + HS detection

In [121]:

%load_ext autoreload
%autoreload 2
from rioplatense_hs.openai import get_completion
from datasets import load_dataset

dataset = load_dataset("piuba-bigdata/contextualized_hate_speech")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [122]:
import json

raw_dataset = json.load(open("../data/dataset.raw.json"))

type(raw_dataset)

list

In [123]:
dataset["test"]

Dataset({
    features: ['id', 'title', 'text', 'context_tweet', 'HATEFUL', 'body', 'CALLS', 'WOMEN', 'LGBTI', 'RACISM', 'CLASS', 'POLITICS', 'DISABLED', 'APPEARANCE', 'CRIMINAL'],
    num_rows: 11343
})

In [124]:
from rioplatense_hs.preprocessing import labels, label_to_text

df_train = dataset["train"].to_pandas()
df_test = dataset["test"].to_pandas()
df_dev = dataset["dev"].to_pandas()
df_train["num_hatred"] = df_train[labels].sum(axis=1)
df_dev["num_hatred"] = df_dev[labels].sum(axis=1)
df_test["num_hatred"] = df_test[labels].sum(axis=1)

df_train["text_label"] = df_train.apply(lambda x: label_to_text(x), axis=1)
df_dev["text_label"] = df_dev.apply(lambda x: label_to_text(x), axis=1)
df_test["text_label"] = df_test.apply(lambda x: label_to_text(x), axis=1)


In [125]:
labels

['WOMEN',
 'LGBTI',
 'RACISM',
 'CLASS',
 'POLITICS',
 'DISABLED',
 'APPEARANCE',
 'CRIMINAL']

In [126]:
df_train[df_train["CRIMINAL"] > 0].sample(2)[["context_tweet", "text"]]

,context_tweet,text
10692,"Coronavirus en Argentina: 341 presos bonaerenses inician huelga de hambre y piden ""irse a sus casas"" https://t.co/GXtHXFL6te",@usuario Claro!!! Así siguen robando y matando... Que se mueran todos
25169,Violencia: creció la cantidad de ladrones abatidos por policías en Buenos Aires https://t.co/A0pOfDGBSF https://t.co/GQDsaWPI2w,@usuario Una buena...


In [127]:
import pandas as pd

pd.options.display.max_colwidth = 1000
#df_train.loc[df_train["LGBTI"] > 0, ["title", "text", "text_label"]].sample(20)

In [129]:
from pprint import pprint as pp
from rioplatense_hs.prompting import build_prompt, examples

print(len(examples))
print(build_prompt(contexto="Estados Unidos restringe los vuelos debido al COVID-19", texto="Todo esto es una locura"))

14
Determinar si el siguiente mensaje contiene discurso de odio. Entendemos que hay discurso de odio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:

- mujer: mujeres o movimiento feminista
- lgbti: contra gays, lesbianas, transexuales y otras identidades de género
- racismo: inmigrantes, xenofobia, o contra pueblos aborígenes
- clase: personas de bajos recursos o motivos de clase
- discapacidad: discapacidad, problemas de salud mental o de adicción al alcohol u otras drogas
- política: ideología política
- aspecto: aspecto, gordofobia o edad
- criminal: presos o delincuentes comunes

Responder una o varias de las características separadas por coma, o bien "nada" si no hay discurso de odio. Pensá paso a paso la respuesta antes de responder.
###
contexto: Al bo

In [130]:
# sample_df = df_test.sample(100)

# sample_df = df_dev.sample(700, random_state=2022)

sample_df = pd.read_csv("../data/sample_dev_df_14_shot.csv")

In [131]:
(sample_df["num_hatred"] > 0).mean()

0.16857142857142857

## Error analysis

In [132]:
from rioplatense_hs.preprocessing import text_to_label, labels
from sklearn.metrics import classification_report

pred_labels = [f"PRED_{label}" for label in labels]
gold_labels = sample_df[labels]
preds = sample_df[pred_labels].astype(int)


print(classification_report(gold_labels, preds, target_names=labels))

              precision    recall  f1-score   support

       WOMEN       0.47      0.33      0.39        21
       LGBTI       0.47      0.69      0.56        13
      RACISM       0.65      0.78      0.70        40
       CLASS       0.21      0.40      0.28        10
    POLITICS       0.24      0.50      0.33        22
    DISABLED       0.33      0.20      0.25         5
  APPEARANCE       0.44      0.36      0.40        22
    CRIMINAL       0.30      0.41      0.35        17

   micro avg       0.41      0.52      0.46       150
   macro avg       0.39      0.46      0.41       150
weighted avg       0.44      0.52      0.46       150
 samples avg       0.10      0.10      0.10       150



/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/jmperez/projects/rioplatense_hate_speech/.venv/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [140]:
pred_labels = [l for l in sample_df.columns if l.startswith("PRED")]

label_hate = sample_df.num_hatred > 0
pred_hate = sample_df[pred_labels].sum(axis=1) > 0

print(classification_report(label_hate, pred_hate))

              precision    recall  f1-score   support

       False       0.92      0.87      0.89       582
        True       0.49      0.64      0.55       118

    accuracy                           0.83       700
   macro avg       0.71      0.75      0.72       700
weighted avg       0.85      0.83      0.84       700



In [142]:
df_test.shape

(11343, 17)

In [102]:
df_train[df_train["HATEFUL"] == 0].sample(10)[["context_tweet", "text"]]

,context_tweet,text
22824,OPINIÓN: Se acaba el tiempo para que el proyecto de Interrupción Voluntaria del Embarazo se trate en el Congreso. Por Mariana Iglesias. https://t.co/ecKOh57RUp,"@usuario Hay prioridades sres.!q se cuiden y chau,siglo XXI y aun no saben como hacerlo!??... #NOALBORTO🙋‍♀️🇦🇷💪🏻"
3013,Las razones de la Justicia para dejar libre al acusado de drogar y violar a la joven venezolana en su primer día de trabajo | Por Leonardo Scannone https://t.co/xeCBm8I0MC,@usuario Que vergüenza por favor. Basta de jueces incompatibles
3206,"Daniel Gollán: ""Con la doctrina de Trump o Bolsonaro, la Argentina tendría entre 25 y 30 mil muertos por coronavirus"" https://t.co/f0G6mMv692",@usuario Pero seríamos o una de las mayores potencias del mundo y con más años de democracia ininterrumpida o el maior do mundo...\nPD.: sin contar que somos aprox 45 millones contra entre 300/250 millones. O sea...
30813,La cruda advertencia de los especialistas: “Si Roldán hubiera matado a su atacante ahora estaría preso” https://t.co/DQ3YJONPIx,"@usuario jjaajajjjaaj si hubiera, imbéciles"
12171,"Julia Mengolini volvió a criticar a Jimena Barón: “Estar buena te sirve solo a vos, no estás haciendo nada por los demás” https://t.co/I7sMgC4qkQ",@usuario @usuario se cree es la dueña de decir que es feminismo y que no. Todo lo que piensan como ella esta bien. No hay actitud mas facho que esa. Mengo dejate el bigote hitleriano
19989,Coronavirus en Argentina: Ginés González García confirmó que traerán médicos cubanos a la Argentina https://t.co/piNZBfPG1w,@usuario @usuario Sigo sin entender para que.
11549,"Santiago Cafiero: “Si se empieza a instalar un discurso de odio, hay que desarmarlo rápidamente” https://t.co/rlYkS4zPTI https://t.co/2jVONdUoQN",@usuario Perdon????? Alguien q le dijo imbecil al dr @usuario habla de discurso de odio? Cuando el es el responsable de publicar a sus hijos?
24924,María Eugenia Vidal: “En Argentina las mujeres somos más que los hombres y debemos tener una representación de acuerdo a ello” https://t.co/CmhB77Xn2P,@usuario Lee el.disparate que decis @usuario !!!!\n\nEl personajillo politico mas sobrevaluado de la historia argentina\n\nDeplorable
15325,"Elisa Carrió, contra Sergio Massa: ""Es un oportunista y un golpista, se merece una denuncia penal por Traición a la Patria"" https://t.co/VjfpDw0wNN",@usuario Psicótica. #RefornaJudicialYa #impuestoALaRiquezaYa #MacriPreso
14020,"El mensaje de Florencia Kirchner antes de volver a la Argentina: ""Tengo lágrimas negras"" https://t.co/bjQdXLpxBI https://t.co/TOx13rGA1q",@usuario Para y por qué volvió?


In [103]:
from rioplatense_hs.prompting import get_response

contexto = 'Julia Mengolini volvió a criticar a Jimena Barón: “Estar buena te sirve solo a vos, no estás haciendo nada por los demás”'
texto = 'se cree es la dueña de decir que es feminismo y que no. Todo lo que piensan como ella esta bien. No hay actitud mas facho que esa. Mengo dejate el bigote hitleriano'

get_response(contexto, texto)

('Determinar si el siguiente mensaje contiene discurso discriminatorio o no. Entendemos que hay discurso discriminatorio en el tweet si tiene declaraciones de carácter intenso e irracional de rechazo, enemistad y aborrecimiento contra un individuo o contra un grupo, siendo estos objetivos de estas expresiones por poseer una característica protegida. Las características protegidas que contemplamos son:\n\n- mujer: Mujeres o movimiento feminista\n- lgbti: Género, identidad sexual\n- racismo: Inmigrantes, extranjeros, religión, pueblos aborígenes u otras nacionalidades\n- clase: Clase, situación socioeconómica o barrio de residencia\n- discapacidad: Discapacidades, problemas de salud mental o de adicción al alcohol u otras drogas\n- política: Opinión o ideología política.\n- aspecto: Aspecto, gordofobia o edad\n- criminal: Antecedentes penales o estar privado de la libertad\n\nResponder una o varias de las características separadas por coma, o bien "nada" si no hay discurso discriminatori

In [137]:
pred_hate.sum()

0

In [135]:
sample_df[pred_hate & ~label_hate][["context_tweet", "text", "text_label", "pred_cot", ]].sample(10)

ValueError: a must be greater than 0 unless no samples are taken

In [78]:
df_train[df_train["num_hatred"] == 0][["context_tweet", "text", "text_label"]].sample(10)

,context_tweet,text,text_label
15354,Loly Antoniale mostró su impresionante casa en Miami: “Soy la reina de mi castillo” https://t.co/GuFd1ythLA,@usuario Mujeres hablando de mujeres 💜,nada
16828,La nueva vida en la cárcel de los rugbiers acusados de matar a Fernando Báez Sosa: comparten todos el mismo celular y leen novelas de fantasía https://t.co/VHfUewstWx,@usuario Uno piensa que solo en Colombia la justicia es ciega \n\nPero con este caso me doy cuenta que es en todo el mundo,nada
27859,"Violó a su nieta, recibió la prisión domiciliaria con la excusa del coronavirus y volvió a contactar a la víctima por teléfono https://t.co/Fgc5jKto1U","@usuario La quería poner el hombre, se solidarizó el juez.",nada
34678,"El equipo médico que venció al coronavirus en China recomendó endurecer la cuarentena en Italia: “Hay demasiada gente en la calle, no sé qué están pensando” https://t.co/CKfSJEuwIL",@usuario A mi me gustaría saber que mierda estaban pensando ellos cuando comían en esos mercados con todos los animales hacinados!,nada
1925,Agrupaciones feministas y la ministra de las Mujeres y Géneros se expresaron contra la liberación de violadores y femicidas https://t.co/ccZOTF32Qp,"@usuario Tu gobierno, querida.",nada
35235,"La asesina serial que mataba hombres para divertirse: sangre, sexo sadomasoquista y excitación en cada puñalada | Por Carolina Balbiani https://t.co/T3EdKxXmkf",@usuario Que rica,nada
25050,Organizaciones sociales cortan la avenida 9 de Julio: reclaman un salario mínimo de $ 45.000 https://t.co/DvQoh4SPaF,@usuario pasaje de ida a Haití y en 10 años somos potencia,nada
26016,"Cuarentena. Carrió: ""Si allanan a Macri, tienen que detener a Alberto Fernández "" https://t.co/GBNki8ew1D https://t.co/tMZwEurVa0",@usuario Bamos LILITA ES ASI,nada
589,Trump volvió a culpar a China por la pandemia y aseguró que “van a pagar un alto precio por lo que le hicieron a EEUU y al mundo” https://t.co/ZvfPtpZNPe,@usuario Te amo,nada
12317,"Juana Viale: ""Me afectó muchísimo que me dijeran golpista "" https://t.co/xNUKlEj9B9 https://t.co/GnejLiVYUB",@usuario Golpista y más aún. Viene de familia (de garcas),nada


In [104]:
# LGBTI errors

label = "CLASS"
sample_df.loc[sample_df[label] != sample_df[f"PRED_{label}"]][["context_tweet", "text", "text_label", "pred_cot"]].sample(5)

,context_tweet,text,text_label,pred_cot
64,🔴 AHORA | Alberto Fernández criticó a Susana Giménez y defendió a Marcelo Tinelli https://t.co/vMUxU3bdKB,@usuario Pregunta... los de la Villa Azul y sus alrededores no son Noticias? No está Berni ahí?,nada,"El texto menciona a la ""Villa Azul"", que es una referencia a un barrio de bajos recursos, y hace alusión a la presencia del Ministro de Seguridad de la provincia de Buenos Aires, Sergio Berni. Por lo tanto, se está haciendo una asociación entre la villa y delincuencia. La respuesta final es ""clase, criminal""."
522,"El drama de una argentina en Bolivia: está embarazada de seis meses, duerme en la calle con su hijo de 4 años y no puede regresar al país | Por Florencia Illbele https://t.co/emWE3AxUqe",@usuario Que feo pensar que acá los bolivianos hasta planes sociales tiene y disfrutan de nuestro sistema de salud y educativo cada vez me convenzo más que los arg somos demasiados generosos,racismo,"El texto alude a los bolivianos y los critica por recibir planes sociales y disfrutar del sistema de salud y educación argentino. La respuesta final es ""racismo, clase""."
382,Coronavirus en Argentina: detienen a dos funcionarios de la Aduana que llevaban dinero de planes sociales argentinos a Paraguay https://t.co/HtxVVz1oZ2,"@usuario AHHHH !!! QUE SON KK ,LOS MUCHACHOS ,,JEJEEJ, ARGENTINA PAIS DE PELOTUDOS /AS..",nada,"El texto utiliza expresiones despectivas y palabras ofensivas para referirse a los funcionarios y a los argentinos en general. La respuesta final es ""clase"""
427,"Relato de un aborto en medio de la desesperación: “Me jugué la vida, cuando no tenés plata te la jugás” | Por Gisele Sousa Dias https://t.co/kFFKKuARi8",@usuario Todo por no ponerse 1 forro! Tomatelas PAYASA,nada,"El texto critica a la persona por no haber utilizado un condón y la insulta llamándola ""payasa"". La respuesta final es ""clase""."
66,"Coronavirus en Argentina: 341 presos bonaerenses inician huelga de hambre y piden ""irse a sus casas"" https://t.co/GXtHXFL6te","@usuario Un gasto menos, dinero q puede ser invertido en salud. Y Si llegan a quemar colchones en señal d protesta como es su costumbre, q duerman en el piso!",nada,"El texto se refiere a los presos como un gasto innecesario y sugiere que si llegan a protestar quemando colchones, deberían dormir en el piso. La respuesta final es ""criminal, clase""."


In [80]:
df_train[(df_train["POLITICS"] > 0) & (df_train["num_hatred"] > 1)][["context_tweet", "text", "text_label"]].sample(5)

,context_tweet,text,text_label
31940,"Grabois defendió la toma de tierras: ""El que dice que ocupar es delito, miente "" https://t.co/qXZsHYnmZ3 https://t.co/R2BLqqV23a",@usuario Yo le pongo una dinamita a estos vagos,"clase, política"
16559,"Le sacó el arma al ladrón que lo asaltó, lo corrió y lo mató de un disparo: quedó detenido https://t.co/fU6UWLDqQe","@usuario Cagones hoy marchen para que suelten a este hombre, no por el hijo de puta de Macri que nos fundió y ayudó a que vuelvan estos negros hijos de puta kirchnerista","racismo, política, criminal"
7604,Santa Fe: rompen más silobolsas y crece la preocupación en el campo https://t.co/Pb1gaBqyXU https://t.co/cPlGjy3V6T,@usuario Donde quedaron los escopetazos Argentina mia?...,"política, criminal"
34614,"Juan Grabois: ""Hay una oleada de toma de tierras y se va a profundizar "" https://t.co/2B1JL2BkHh https://t.co/jJXbT0X564","@usuario Si es una amenaza!!!!??!!...a Vos hdrmPtas, seras otro de los tantos que vamos a LINCHAR en la Plaza de Mayo!!!!","clase, política"
21056,"“Tenemos explotados los teléfonos con denuncias por gente que no cumple la cuarentena”, aseguró la titular de Migraciones | Por Hugo Martin https://t.co/IJgFwW4Gbk","@usuario Se supone q es gente escolarizada, con educación, después dicen peronchos! Hdp","racismo, política"


In [91]:
sample_df["PRED_HATEFUL"] = pred_hate.astype(int)

In [92]:
sample_df.to_csv("../data/sample_dev_df.csv", index=False)